In [1]:
# Import Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count

In [2]:
# Create Spark Session
spark = SparkSession.builder.appName("TitanicDataAnalysis").getOrCreate()

In [3]:
# Load Dataset (Replace with your file path or upload to Colab)
from google.colab import files
files.upload()  # Upload 'titanic.csv'
df = spark.read.csv("/content/titanic.csv", header=True, inferSchema=True)

Saving titanic.csv to titanic (1).csv


In [4]:
# Basic Data Exploration
print("Schema:")
df.printSchema()
print(f"Total Rows: {df.count()}")
df.show(5)

Schema:
root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

Total Rows: 891
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|fem

In [5]:
# Count Missing Data
print("Missing Data:")
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

Missing Data:
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [6]:
# Data Cleaning
df = df.na.fill({"Age": 30, "Embarked": "S"}).withColumn("Age", col("Age").cast("float")).withColumn("Fare", col("Fare").cast("float"))

In [7]:
# Analysis: Survival Rate
print("Survival Rate:")
df.groupBy("Survived").count().show()

Survival Rate:
+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [8]:
# Survival Rate by Gender
print("Survival Rate by Gender:")
df.groupBy("Sex", "Survived").count().show()

Survival Rate by Gender:
+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



In [9]:
# Average Fare by Passenger Class
print("Average Fare by Pclass:")
df.groupBy("Pclass").avg("Fare").show()

Average Fare by Pclass:
+------+------------------+
|Pclass|         avg(Fare)|
+------+------------------+
|     1| 84.15468752825701|
|     3|13.675550210257411|
|     2| 20.66218318109927|
+------+------------------+



In [10]:
# Correlation Between Age and Fare
print(f"Correlation between Age and Fare: {df.stat.corr('Age', 'Fare')}")

Correlation between Age and Fare: 0.09063186984336771


In [11]:
# Top 5 Most Expensive Tickets
print("Top 5 Most Expensive Tickets:")
df.orderBy(col("Fare").desc()).show(5)

Top 5 Most Expensive Tickets:
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|        259|       1|     1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|       NULL|       C|
|        680|       1|     1|Cardeza, Mr. Thom...|  male|36.0|    0|    1|PC 17755|512.3292|B51 B53 B55|       C|
|        738|       1|     1|Lesurer, Mr. Gust...|  male|35.0|    0|    0|PC 17755|512.3292|       B101|       C|
|         89|       1|     1|Fortune, Miss. Ma...|female|23.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|
|         28|       0|     1|Fortune, Mr. Char...|  male|19.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|
+-----------+--------+------+--------------------+------+-

In [12]:
# Save Processed Data
df.write.csv("processed_titanic_data.csv", header=True)
print("Processed data saved!")

Processed data saved!


In [13]:
df.groupBy("Pclass", "Survived").count().show()

+------+--------+-----+
|Pclass|Survived|count|
+------+--------+-----+
|     1|       0|   80|
|     3|       1|  119|
|     1|       1|  136|
|     2|       1|   87|
|     2|       0|   97|
|     3|       0|  372|
+------+--------+-----+



In [14]:
# Survival Rate by Age Group
df = df.withColumn("Age_Group", when(col("Age") < 18, "Child").when((col("Age") >= 18) & (col("Age") <= 60), "Adult").otherwise("Senior"))
df.groupBy("Age_Group", "Survived").count().show()

+---------+--------+-----+
|Age_Group|Survived|count|
+---------+--------+-----+
|    Adult|       0|  480|
|    Adult|       1|  276|
|   Senior|       0|   17|
|   Senior|       1|    5|
|    Child|       1|   61|
|    Child|       0|   52|
+---------+--------+-----+



In [15]:
# Survival Rate by Embarked Port
df.groupBy("Embarked", "Survived").count().show()

+--------+--------+-----+
|Embarked|Survived|count|
+--------+--------+-----+
|       Q|       1|   30|
|       S|       0|  427|
|       S|       1|  219|
|       C|       1|   93|
|       Q|       0|   47|
|       C|       0|   75|
+--------+--------+-----+



In [16]:
# Gender vs Survival
df.groupBy("Sex", "Survived").count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



In [17]:
# Average Age and Fare by Survival Status
df.groupBy("Survived").avg("Age", "Fare").show()

+--------+------------------+------------------+
|Survived|          avg(Age)|         avg(Fare)|
+--------+------------------+------------------+
|       1|28.595526315751133|48.395407697610686|
|       0|30.483606557377048| 22.11788694367817|
+--------+------------------+------------------+



In [18]:
# Correlation Between Age and Fare
print(f"Correlation between Age and Fare: {df.stat.corr('Age', 'Fare')}")


Correlation between Age and Fare: 0.09063186984336771


In [20]:
# Top 5 Most Expensive Tickets
df.orderBy(col("Fare").desc()).show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+---------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|      Cabin|Embarked|Age_Group|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+---------+
|        259|       1|     1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|       NULL|       C|    Adult|
|        680|       1|     1|Cardeza, Mr. Thom...|  male|36.0|    0|    1|PC 17755|512.3292|B51 B53 B55|       C|    Adult|
|        738|       1|     1|Lesurer, Mr. Gust...|  male|35.0|    0|    0|PC 17755|512.3292|       B101|       C|    Adult|
|         89|       1|     1|Fortune, Miss. Ma...|female|23.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|    Adult|
|         28|       0|     1|Fortune, Mr. Char...|  male|19.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|    Adult|
+-------